In [ ]:
import pyomo.environ as pyo
import idaes
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock)
import idaes.core.util.scaling as iscale
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop
from idaes.models_extra.power_generation.unit_models.soec_design import SoecDesign, EosType
import pytest


In [ ]:

def soec_example_flowsheet(eos=EosType.PR):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    sweep_comp = {
        "O2":0.2074,
        "H2O":0.0099,
        "CO2":0.0003,
        "N2":0.7732,
        "Ar":0.0092,
    }

    feed_comp = {
        "H2": 0.01,
        "H2O": 0.99,
    }

    m.fs.o2_side_prop_params = GenericParameterBlock(
        **get_prop(sweep_comp, {"Vap"}, eos=eos),
        doc="Air property parameters",
    )
    m.fs.h2_side_prop_params = GenericParameterBlock(
        **get_prop(feed_comp, {"Vap"}, eos=eos),
        doc="Flue gas property parameters",
    )
    m.fs.soec = SoecDesign(
        oxygen_side_property_package=m.fs.o2_side_prop_params,
        hydrogen_side_property_package=m.fs.h2_side_prop_params,
        reaction_eos=eos
    )

    m.fs.soec.hydrogen_side_inlet.temperature.fix(1023)
    m.fs.soec.hydrogen_side_inlet.pressure.fix(20e5)
    m.fs.soec.hydrogen_side_inlet.flow_mol.fix(2)
    for (t, i), c in m.fs.soec.hydrogen_side_inlet.mole_frac_comp.items():
        c.fix(feed_comp[i])
    m.fs.soec.oxygen_side_inlet.temperature.fix(1023)
    m.fs.soec.oxygen_side_inlet.pressure.fix(20e5)
    m.fs.soec.oxygen_side_inlet.flow_mol.fix(2)
    for (t, i), c in m.fs.soec.oxygen_side_inlet.mole_frac_comp.items():
        c.fix(sweep_comp[i])
    m.fs.soec.hydrogen_side_outlet_temperature.fix(1023)
    m.fs.soec.oxygen_side_outlet_temperature.fix(1023)
    m.fs.soec.water_utilization.fix(0.7)
    iscale.calculate_scaling_factors(m)
    m.fs.soec.initialize(optarg={"max_iter":30})
    return m

# create a flowsheet with the SOEC model
model = soec_example_flowsheet()

In [ ]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 

In [ ]:
# Visualize the model
visualize(model.fs, "SOEC")